<a href="https://colab.research.google.com/github/anko191/Python_Kaggle/blob/master/XGBoost/XGboost_several_parameter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gradient Boosting
* At first, naive model -> make predictions
* for loop ->
    * make predictions
    * calculate loss
    * train new model
    * add new model to ensemble
    * repeat!

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv('/content/melb_data.csv')

cols_to_use = ['Rooms', 'Distance', 'Landsize', 'BuildingArea', 'YearBuilt']
X = data[cols_to_use]

y = data.Price

X_train, X_valid, y_train, y_valid = train_test_split(X, y)

# XGboost は extreme gradient boosting の略

In [2]:
from xgboost import XGBRegressor
my_model = XGBRegressor()
my_model.fit(X_train, y_train)

[00:50:19] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [4]:
from sklearn.metrics import mean_absolute_error
predictions = my_model.predict(X_valid)
print("Mean Absolute Error: " + str(mean_absolute_error(predictions, y_valid)))

Mean Absolute Error: 270378.40682529454


# XGBoost Parameter Tuning
以下にパラメータ全部載せるぞ

## n_estimators
* これは何回行うか、アンサンブルに含めるモデルの数と等しくなるらしい
    * 値が低すぎるとアンダーフィッティングとなり、訓練データとテストデータの両方で不正確な予測を引き起こす
    * 値が高すぎるとオーバーフィッティングになり、訓練データでは正確な予測が出来ますが、<b>テストデータでは</b>不正確な予測になる 一番大事
    * 典型的なのは<b>100 から 1000</b>
    * learning_rate パラメータに大きく依存します

## early_stopping_rounds
* n_estimatorsの理想的な値を自動的に見つける方法
    * 検証スコアの改善が止まると、モデルの反復処理を停止させる
    * n_estimators に高い値を設定し、early_stopping_roundsを使用して反復処理を停止する最適な時間を見つけるのが賢い方法
    * ランダムによって、検証スコアが改善されないラウンドが1回だけはっせいすることがあるので、停止する前に、ストレートな劣化を何ラウンド許すかを数値で指定する必要があります。！
    * = 5が妥当だよ！ 検証スコアが5ラウンド連続で悪化した後に停止します

In [ ]:
my_model = XGBRegressor(n_estimators = 500)
my_model.fit(X_train, y_train,
             early_stopping_rounds = 5,
             eval_set = [(X_valid, y_valid)],
             verbose = True)

## learning_rate
* 予測値に小さな数をかけてから追加することで、アンサンブルに追加する各ツリーの助けが少なくなることを意味します。そのため、オーバーフィットすることなく、n_estimatorsの値を高く設定することが出来ます

In [ ]:
my_model = XGBRegressor(n_estimators = 1000, learning_rate = 0.05)
my_model.fit(X_train, y_train,
             early_stopping_rounds = 5,
             eval_set=[(X_valid, y_valid)],
             verbose = True)

## n_jobs
* 並列性を追加する、大規模なデータセットに有向
    * n_jobsをマシンのコア数と等しく設定するのが一般的
    * まあ正直意味ないけど時間は短くなるからこれはマジでやれ
    

In [ ]:
my_model = XGBRegressor(n_estimators = 1000, learning_rate = 0.05, n_jobs = 4)
my_model.fit(X_train, y_train,
             early_stopping_rounds = 5,
             eval_set=[(X_valid, y_valid)],
             verbose = True)

In [15]:
import pprint
pprint.pprint("base_score=0.5"""", booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.05, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=1000,
             n_jobs=4, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1
             """)

("base_score=0.5, booster='gbtree', colsample_bylevel=1,\n"
 '             colsample_bynode=1, colsample_bytree=1, gamma=0,\n'
 "             importance_type='gain', learning_rate=0.05, max_delta_step=0,\n"
 '             max_depth=3, min_child_weight=1, missing=None, '
 'n_estimators=1000,\n'
 "             n_jobs=4, nthread=None, objective='reg:linear', "
 'random_state=0,\n'
 '             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,\n'
 '             silent=None, subsample=1, verbosity=1\n'
 '             ')


In [ ]:
my_model = XGBRegressor(n_estimators = 1000, learning_rate = 0.05, n_jobs = 4,max_depth = 6)
my_model.fit(X_train, y_train,
             early_stopping_rounds = 5,
             eval_set=[(X_valid, y_valid)],
             verbose = True)